In [2]:
## 获得视频教程
from onekey_algo.custom.Manager import onekey_show
onekey_show('Sol21-Habitat', force_show=True)

Habitat Radiomics and Deep Learning Features Based on CT for Predicting Lymphovascular Invasion in T1-stage Lung Adenocarcinoma: A Multicenter Study

- 生境分析 | 临床的立意【为什么要！】

  - 目前经典思路：ROI作为一个整体
    - 1. 传统组学
    - 2. 2D深度学习 | ROI最大截面
    - 3. 3D深度学习 | VOI区域，最小外接立方体
    - 4. 2.5D深度学习 | 若干Slice
    - ...
  - 囫囵吞枣，眉毛胡子一把抓 -> 临床：异质性 | 肿瘤微环境
    - ROI的不同区域，不同细胞聚集的区域 | 它对你的研究的问题影响不尽相同
    - 想法：对不同的区域，进行单独的研究。
    - 期望：不同区域有个结果，对比研究不同区域对我研究问题的差异性；区域融合有个结果，对比融合跟单独区域以及ROI整体区域的差异性
      - 1. 区域的对比
      - 2. 整体 vs 细致刻画融合

- 生境在Onekey中的实现过程

  - Step0. 收集整理数据（转成nii.gz）

    - 建议：所有半路出家的（你的数据不是原生通过Onekey转成的nii.gz）-> 再用Onekey工具转化一下

  - Step1. ROI的勾画

    - 1. Onekey-Comp0组件
    - 2. 人工勾画

  - Step2. 通过某种方法，将ROI划分成不同的亚区

    - 1. 手工画 | 脑胶质瘤 -> 肿瘤，水肿，坏死……

    - 2. 阈值法，OTSU | 序列多的时候 | 不建议

      - 假设：3个序列
      - Step1. 每个序列进行进行OTSU，划分成2个区域
      - Step2. 对每个区域里的【体素】进行二进制编码 -> 一个对应位置的【体素】，8种状态 | e.g. 100
      - 弊端
        - 1. 生境区域数没法控制，大的没边，小到没有
        - 2. 依赖于图像的配准
        - 3. 只有一个序列的情况

    - 3. 无监督聚类，体素【物以类聚人以群分】

      - Step1. 对研究的对象【体素】进行特征的刻画，特征越多，刻画越精细，理论来说聚类效果越好

        - 1. 你自己本身啥样 -> 体素值

        - 2. 你跟谁玩 -> 体素周围的体素啥样

          - --whole, 可视化视野的大小
          - --mask_id_spec，指定用那个ROI区域进行特征刻画
          - 前方高能：默认功能，对算力（能花多少花多少 | 9950x3d，有能力直接上服务器） | 存储 要求极高！ 经验：多机并行~
          - --mode，局部特征刻画的模式
            - quality，质量优先，每个体素通过周围5*5*5共同决定
            - fast，速度优先，通过周围的1+1+1+1+1+1体素决定 | 理论速度是quality的20x
            - 张老师坚信：能用钱解决的问题就不是问题
          - -j，并行度，最大程度压榨你的CPU计算资源， < #CPU/4
          - --param_file，你要提取的特征是什么，跟组学配置方式完全一样 | 特征越多肯定越慢 | 尽量复制到一个没有中文，没有空格的地方，好比说桌面
          - --skip_threshold：体素数大于多少的时候，跳过
          - 时间估计：quality模式，i9 | 9950x CPU，300个样本，ROI大小1w左右的体素，大概时间30天

      - Step2. 聚类的方法将【体素】分组

        - --method 指定使用的聚类方法，默认kmeans | 20250615发布
        - --cluster，聚类中心数据，建议大家多选几个
          - 1. 如果你只使用体素值聚类，随聚类数增加，单调递增 -> 老老实实使用局部特征刻画
          - 2. 局部特征刻画，随聚类数增加，单调递减！从3类开始聚类
          - 最好的聚类中心数：3-5
        - --mask_id_spec，与roi特征提取的参数一致
        - --separate， 是否每个样本单独做生境聚类
          - 不能保证，每个样本，有所有的聚类亚区。
          - separate可以，不能保证每个样本的亚区具有相同的物理含义
          - 延伸阅读：生境最后一公里！ www.bilibili.com/video/BV16BeFe2Eyd
        - 时间消耗：直接跟你的硬盘息息相关 | 小时为单位解决

  - Step3. 为所欲为！

- 生境的奇技淫巧

  - 1. 如何让特征刻画跑的更快 时间 = 需要处理的【体素】量 ÷ 单位时间的处理速度 体素量 = 样本数 × 每个样本的体素量

    - 1. -j并行度，必要的时候多开

    - 2. mode，必要的时候使用fast模式。

    - 3. 多机并行

    - 4. 考虑采样的；1*1*1 有2700个体素 -> 3 * 3 * 3 100个体素

      - 推荐使用场景：碰到ROI特别大的时候
      - 经验：张老师一般采样通道维度！1 1 5
        - --new_res 1 1 5

    - 马儿跑得快！还不想让他吃草！-> 刻画会很模糊！

      - 越偷懒，效果一般越差
      - 非必要，不要在精度上进行损失！

- 答疑汇总

  - 1. 超声的数据，能不能做生境

    - 所有的数据，任何场景，只要你吧数据转化成了nii.gz；生境的处理过程都OK！
    - “生境”这个故事在你的场景中适不适用，自己考虑！
    - 前提：只要你能画出来ROI！

  - 2. 为什么是这19个特征？

    - 跟着感觉走~
    - 你觉得不行，可以自己感觉~

  - 3. 如果你一次跑不完所有的样本，记得下次跑的时候，一定注意使用相同的配置文件！

    - 如果出现不匹配，记得在viz找到不匹配的样本，删掉不符合预期的npy文件即可！

  - 4. 数据量

    - 生境分析，目前对样本量容忍程度最高
    - 比较适合，样本比较少的场景！

In [1]:
## 获得视频教程
from onekey_algo.custom.Manager import onekey_show
onekey_show('Sol21-Habitat-Spec', force_show=True)